In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
from __future__ import print_function
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
from torch import nn
from tqdm.auto import tqdm#show loop progress
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import itertools
import torchvision.models as models


import argparse
import os

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import torchvision.models as models
import matplotlib.animation as animation
from IPython.display import HTML

from scipy import linalg
from torch.nn.functional import adaptive_avg_pool2d

from PIL import Image

import matplotlib.pyplot as plt
import sys
import numpy as np
import os
# print(os.listdir("../input"))

import time
from torchvision.models.inception import inception_v3
import numpy as np
from scipy.stats import entropy
from torch.autograd import Variable
from torch.nn import functional as F
import torch.utils.data
import torch
from torch import nn

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
device = 'cuda'
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cnn-c1/cnn_c1(2).pt
/kaggle/input/mnist-splitter/stdash_dataloader
/kaggle/input/mnist-splitter/svdash_dataloader
/kaggle/input/mnist-splitter-newtensor/__results__.html
/kaggle/input/mnist-splitter-newtensor/MNIST.tar.gz
/kaggle/input/mnist-splitter-newtensor/svload
/kaggle/input/mnist-splitter-newtensor/stload
/kaggle/input/mnist-splitter-newtensor/__notebook__.ipynb
/kaggle/input/mnist-splitter-newtensor/__output__.json
/kaggle/input/mnist-splitter-newtensor/custom.css
/kaggle/input/mnist-splitter-newtensor/MNIST/processed/training.pt
/kaggle/input/mnist-splitter-newtensor/MNIST/processed/test.pt
/kaggle/input/mnist-splitter-newtensor/MNIST/raw/t10k-labels-idx1-ubyte
/kaggle/input/mnist-splitter-newtensor/MNIST/raw/t10k-images-idx3-ubyte.gz
/kaggle/input/mnist-splitter-newtensor/MNIST/raw/t10k-labels-idx1-ubyte.gz
/kaggle/input/mnist-splitter-newtensor/MNIST/raw/train-labels-idx1-ubyte.gz
/kaggle/input/mnist-splitter-newtensor/MNIST/raw/t10k-images-idx3-ubyte
/kaggle/i

In [2]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms
from torch import nn, optim

transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])
cnn_c1 = torch.load('../input/cnn-c1/cnn_c1(2).pt')
cnn_c1.to('cuda')
cnn_c1.eval()

Sequential(
  (0): Linear(in_features=784, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=64, bias=True)
  (3): Tanh()
  (4): Linear(in_features=64, out_features=10, bias=True)
  (5): LogSoftmax(dim=1)
)

In [3]:
def find_data(trainloader,epoch):
    correct_count, all_count = 0, 0
    predicted_labels = []
    data = []
    for images in trainloader:
      for i in range(len(images)):
        #print(images[i].size())
        img = images[i].view(1, 784)

        with torch.no_grad():
            logps = cnn_c1(img)

        ps = torch.exp(logps)
        probab = list(ps.cpu().numpy()[0])
        pred_label = probab.index(max(probab))
        predicted_labels.append(pred_label)
        data.append([images[i],pred_label])
    len(data)
    gan_testloader = torch.utils.data.DataLoader(data, batch_size=64, shuffle=True)
    torch.save(gan_testloader, './gan_data_epoch_{}'.format(epoch))


In [4]:
class InceptionV3(nn.Module):
    """Pretrained InceptionV3 network returning feature maps"""

    # Index of default block of inception to return,
    # corresponds to output of final average pooling
    DEFAULT_BLOCK_INDEX = 3

    # Maps feature dimensionality to their output blocks indices
    BLOCK_INDEX_BY_DIM = {
        64: 0,   # First max pooling features
        192: 1,  # Second max pooling featurs
        768: 2,  # Pre-aux classifier features
        2048: 3  # Final average pooling features
    }

    def __init__(self,
                 output_blocks=[DEFAULT_BLOCK_INDEX],
                 resize_input=True,
                 normalize_input=True,
                 requires_grad=False):
        
        super(InceptionV3, self).__init__()

        self.resize_input = resize_input
        self.normalize_input = normalize_input
        self.output_blocks = sorted(output_blocks)
        self.last_needed_block = max(output_blocks)

        assert self.last_needed_block <= 3, \
            'Last possible output block index is 3'

        self.blocks = nn.ModuleList()

        
        inception = models.inception_v3(pretrained=True)

        # Block 0: input to maxpool1
        block0 = [
            inception.Conv2d_1a_3x3,
            inception.Conv2d_2a_3x3,
            inception.Conv2d_2b_3x3,
            nn.MaxPool2d(kernel_size=3, stride=2)
        ]
        self.blocks.append(nn.Sequential(*block0))

        # Block 1: maxpool1 to maxpool2
        if self.last_needed_block >= 1:
            block1 = [
                inception.Conv2d_3b_1x1,
                inception.Conv2d_4a_3x3,
                nn.MaxPool2d(kernel_size=3, stride=2)
            ]
            self.blocks.append(nn.Sequential(*block1))

        # Block 2: maxpool2 to aux classifier
        if self.last_needed_block >= 2:
            block2 = [
                inception.Mixed_5b,
                inception.Mixed_5c,
                inception.Mixed_5d,
                inception.Mixed_6a,
                inception.Mixed_6b,
                inception.Mixed_6c,
                inception.Mixed_6d,
                inception.Mixed_6e,
            ]
            self.blocks.append(nn.Sequential(*block2))

        # Block 3: aux classifier to final avgpool
        if self.last_needed_block >= 3:
            block3 = [
                inception.Mixed_7a,
                inception.Mixed_7b,
                inception.Mixed_7c,
                nn.AdaptiveAvgPool2d(output_size=(1, 1))
            ]
            self.blocks.append(nn.Sequential(*block3))

        for param in self.parameters():
            param.requires_grad = requires_grad

    def forward(self, inp):
        """Get Inception feature maps
        Parameters
        ----------
        inp : torch.autograd.Variable
            Input tensor of shape Bx3xHxW. Values are expected to be in
            range (0, 1)
        Returns
        -------
        List of torch.autograd.Variable, corresponding to the selected output
        block, sorted ascending by index
        """
        outp = []
        x = inp

        if self.resize_input:
            x = F.interpolate(x,
                              size=(299, 299),
                              mode='bilinear',
                              align_corners=False)

        if self.normalize_input:
            x = 2 * x - 1  # Scale from range (0, 1) to range (-1, 1)

        for idx, block in enumerate(self.blocks):
            x = block(x)
            if idx in self.output_blocks:
                outp.append(x)

            if idx == self.last_needed_block:
                break

        return outp
    
block_idx = InceptionV3.BLOCK_INDEX_BY_DIM[2048]
model = InceptionV3([block_idx])
model=model.cuda()


def calculate_activation_statistics(images,model,batch_size=128, dims=2048,
                    cuda=False):
    model.eval()
    act=np.empty((len(images), dims))
    
    if cuda:
        batch=images.cuda()
    else:
        batch=images
    pred = model(batch)[0]

        # If model output is not scalar, apply global spatial average pooling.
        # This happens if you choose a dimensionality not equal 2048.
    if pred.size(2) != 1 or pred.size(3) != 1:
        pred = adaptive_avg_pool2d(pred, output_size=(1, 1))

    act= pred.cpu().data.numpy().reshape(pred.size(0), -1)
    
    mu = np.mean(act, axis=0)
    sigma = np.cov(act, rowvar=False)
    return mu, sigma
def calculate_frechet_distance(mu1, sigma1, mu2, sigma2, eps=1e-6):
    """Numpy implementation of the Frechet Distance.
    The Frechet distance between two multivariate Gaussians X_1 ~ N(mu_1, C_1)
    and X_2 ~ N(mu_2, C_2) is
            d^2 = ||mu_1 - mu_2||^2 + Tr(C_1 + C_2 - 2*sqrt(C_1*C_2)).
    """

    mu1 = np.atleast_1d(mu1)
    mu2 = np.atleast_1d(mu2)

    sigma1 = np.atleast_2d(sigma1)
    sigma2 = np.atleast_2d(sigma2)

    assert mu1.shape == mu2.shape, \
        'Training and test mean vectors have different lengths'
    assert sigma1.shape == sigma2.shape, \
        'Training and test covariances have different dimensions'

    diff = mu1 - mu2

    
    covmean, _ = linalg.sqrtm(sigma1.dot(sigma2), disp=False)
    if not np.isfinite(covmean).all():
        msg = ('fid calculation produces singular product; '
               'adding %s to diagonal of cov estimates') % eps
        print(msg)
        offset = np.eye(sigma1.shape[0]) * eps
        covmean = linalg.sqrtm((sigma1 + offset).dot(sigma2 + offset))

    
    if np.iscomplexobj(covmean):
        if not np.allclose(np.diagonal(covmean).imag, 0, atol=1e-3):
            m = np.max(np.abs(covmean.imag))
            raise ValueError('Imaginary component {}'.format(m))
        covmean = covmean.real

    tr_covmean = np.trace(covmean)

    return (diff.dot(diff) + np.trace(sigma1) +
            np.trace(sigma2) - 2 * tr_covmean)

def calculate_fretchet(images_real,images_fake,model):
     mu_1,std_1=calculate_activation_statistics(images_real,model,cuda=True)
     mu_2,std_2=calculate_activation_statistics(images_fake,model,cuda=True)
    
     """get fretched distance"""
     fid_value = calculate_frechet_distance(mu_1, std_1, mu_2, std_2)
     return fid_value
    

from torchvision.models.inception import inception_v3
import numpy as np
from scipy.stats import entropy
from torch.autograd import Variable
from torch.nn import functional as F
import torch.utils.data
import torch
from torch import nn

def inception_score(imgs, cuda=True, batch_size=1, resize=False, splits=1):
    """Computes the inception score of the generated images imgs
    imgs -- Torch dataset of (3xHxW) numpy images normalized in the range [-1, 1]
    cuda -- whether or not to run on GPU
    batch_size -- batch size for feeding into Inception v3
    splits -- number of splits
    """
    N = len(imgs)

    assert batch_size > 0
    assert N > batch_size

    # Set up dtype
    if cuda:
        dtype = torch.cuda.FloatTensor
    else:
        if torch.cuda.is_available():
            print("WARNING: You have a CUDA device, so you should probably set cuda=True")
        dtype = torch.FloatTensor

    # Set up dataloader
    dataloader = torch.utils.data.DataLoader(imgs, batch_size=batch_size)
    
    # Load inception model
    inception_model = inception_v3(pretrained=True, transform_input=False).type(dtype)
    inception_model.eval();
    up = nn.Upsample(size=(299, 299), mode='bilinear').type(dtype)
    def get_pred(x):
        if resize:
            x = up(x)
        x = inception_model(x)
        return F.softmax(x).data.cpu().numpy()

    # Get predictions
    preds = np.zeros((N, 1000))

    for i, batch in enumerate(dataloader, 0):
        batch = batch.type(dtype)
        batchv = Variable(batch)
        batch_size_i = batch.size()[0]

        preds[i*batch_size:i*batch_size + batch_size_i] = get_pred(batchv)

    # Now compute the mean kl-div
    split_scores = []

    for k in range(splits):
        part = preds[k * (N // splits): (k+1) * (N // splits), :]
        py = np.mean(part, axis=0)
        scores = []
        for i in range(part.shape[0]):
            pyx = part[i, :]
            scores.append(entropy(pyx, py))
        split_scores.append(np.exp(np.mean(scores)))

    return np.mean(split_scores), np.std(split_scores)

def PSNR(original, compressed): 
    mse = torch.mean((original - compressed) ** 2) 
    if(mse == 0):  # MSE is zero means no noise is present in the signal . 
                  # Therefore PSNR have no importance. 
        return 100
    max_pixel = 255.0
    psnr = 20 * torch.log10(max_pixel / torch.sqrt(mse)) 
    return psnr 
"""Emprical maximum mean discrepancy. The lower the result
       the more evidence that distributions are the same.

    Args:
        x: first sample, distribution P
        y: second sample, distribution Q
        kernel: kernel type such as "multiscale" or "rbf"
"""
def MMD(x, y, kernel):
    print(x.size())
    print(y.size())
    x = x.reshape(x.size(0), x.size(2) * x.size(3))
    y = y.reshape(y.size(0), y.size(2) * y.size(3))
    print(x.size())
    print(y.size())

    xx, yy, zz = torch.mm(x, x.t()), torch.mm(y, y.t()), torch.mm(x, y.t())
    rx = (xx.diag().unsqueeze(0).expand_as(xx))
    ry = (yy.diag().unsqueeze(0).expand_as(yy))
    
    dxx = rx.t() + rx - 2. * xx # Used for A in (1)
    dyy = ry.t() + ry - 2. * yy # Used for B in (1)
    dxy = rx.t() + ry - 2. * zz # Used for C in (1)
    
    XX, YY, XY = (torch.zeros(xx.shape).to(device),
                  torch.zeros(xx.shape).to(device),
                  torch.zeros(xx.shape).to(device))
    
    if kernel == "multiscale":
        
        bandwidth_range = [0.2, 0.5, 0.9, 1.3]
        for a in bandwidth_range:
            XX += a**2 * (a**2 + dxx)**-1
            YY += a**2 * (a**2 + dyy)**-1
            XY += a**2 * (a**2 + dxy)**-1
            
    if kernel == "rbf":
      
        bandwidth_range = [10, 15, 20, 50]
        for a in bandwidth_range:
            XX += torch.exp(-0.5*dxx/a)
            YY += torch.exp(-0.5*dyy/a)
            XY += torch.exp(-0.5*dxy/a)
      
      

    return torch.mean(XX + YY - 2. * XY)


Downloading: "https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-1a9a5a14.pth


  0%|          | 0.00/104M [00:00<?, ?B/s]

In [5]:
from six.moves import urllib
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

In [6]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms
from torch import nn, optim
%matplotlib inline

# transform = transforms.Compose([transforms.Resize(28),transforms.ToTensor(),
#                               transforms.Normalize((0.5,), (0.5,)),
#                               ])

# trainset = datasets.MNIST(r'..\input\MNIST', download=True, train=True, transform=transform)#(St')
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# testset = datasets.MNIST(r'..\input\MNIST', download=True, train=False, transform=transform)#(Sv')
# testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)



In [7]:
# final_set = trainset+ testset

In [8]:
# train_size = int(0.8571 * len(final_set))
# test_size = len(final_set) - train_size
# train_dataset, test_dataset = torch.utils.data.random_split(final_set, [train_size, test_size])

In [9]:
train_loader = torch.load('../input/mnist-splitter-newtensor/stload')
test_loader = torch.load('../input/mnist-splitter-newtensor/svload')
len(train_loader)

938

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
%matplotlib inline

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.fc_1 = nn.Linear(74, 1024)
        self.fc_2 = nn.Linear(1024, 7*7*128)

        self.bn_1 = nn.BatchNorm1d(1024)
        self.bn_2 = nn.BatchNorm2d(128)
        self.bn_3 = nn.BatchNorm2d(64)

        self.upconv_1 = nn.ConvTranspose2d(128, 64, (4,4), stride=2, padding=1, bias=False)
        self.upconv_2 = nn.ConvTranspose2d(64, 1, (4,4), stride=2, padding=1, bias=False)

    def forward(self,x):
        # Construct network described in paper
        x = self.bn_1(F.relu(self.fc_1(x)))
        x = self.fc_2(x)
        x = x.view(-1,128,7,7)
        x = self.bn_2(F.relu(x))
        x = self.bn_3(F.relu(self.upconv_1(x)))
        x = F.sigmoid(self.upconv_2(x))
        return x

class DiscriminatorFrontEnd(nn.Module):
    def __init__(self):
        super(DiscriminatorFrontEnd, self).__init__()

        self.fc = nn.Linear(7*7*128, 1024)

        self.bn_1 = nn.BatchNorm2d(128)
        self.bn_2 = nn.BatchNorm1d(1024)

        self.conv_1 = nn.Conv2d(1, 64, (4,4), stride=2, padding=1, bias=False)
        self.conv_2 = nn.Conv2d(64, 128, (4,4), stride=2, padding=1, bias=False)

    def forward(self,x):
        # Construct network described in paper
        # Input = 1x28x28
        x = F.leaky_relu(self.conv_1(x))
        # x.shape = 64x14x14
        x = F.leaky_relu(self.conv_2(x))
        # x.shape = 128x7x7
        x = self.bn_1(x)
        x = x.view(-1,7*7*128)
        x = self.fc(x)
        x = self.bn_2(x)
        return x

class DiscriminatorBackend(nn.Module):
    def __init__(self):
        super(DiscriminatorBackend, self).__init__()
        self.fc = nn.Linear(1024, 1)
    def forward(self,x):
        x = F.sigmoid(self.fc(x))
        return x

class DiscriminatorInfo(nn.Module):
    def __init__(self):
        super(DiscriminatorInfo, self).__init__()
        self.fc_1 = nn.Linear(1024, 128)
        self.fc_2 = nn.Linear(128,12)

        self.bn = nn.BatchNorm1d(128)

    def forward(self,x):
        x = self.fc_1(x)
        x = F.leaky_relu(self.bn(x))
        x = self.fc_2(x)
        return x
import numpy as np
import torch
import matplotlib.pyplot as plt

def sample_noise(batch_size, num_category = 10, num_conti=2, noise_dim=62):
    idx = np.random.randint(num_category, size=batch_size)
    category_code = np.zeros((batch_size, num_category))
    category_code[range(batch_size),idx] = 1.0
    conti_code = np.random.uniform(-1.0,1.0,(batch_size,num_conti))

    random_noise = np.random.uniform(-1.0,1.0,(batch_size,noise_dim))

    z = torch.cat((torch.Tensor(random_noise),torch.Tensor(category_code),torch.Tensor(conti_code)),dim=1)

    return z, idx

def get_test_noise(num_category = 10, num_conti=2, noise_dim=62):
    fixed_noise = np.random.uniform(-1.0,1.0,(noise_dim))
    # z1 : fix c2
    # z2 : fix c1
    z1 = []
    z2 = []
    for cat in range(num_category):
        category_code = np.zeros((num_category))
        category_code[cat] = 1.0
        for c in np.arange(-2,2.1,0.5):
            z1.append(np.concatenate([fixed_noise,category_code,np.array([c]),np.array([0])]))
            z2.append(np.concatenate([fixed_noise,category_code,np.array([0]),np.array([c])]))
    z1 = torch.Tensor(np.array(z1))
    z2 = torch.Tensor(np.array(z2))

    return z1,z2

def save_fig(z,G,fig_name,num_category = 10,num_conti=9):
    fake_x = G(z).data.cpu().numpy().reshape(num_category*num_conti,28,28)
    fig, axs = plt.subplots(num_category,num_conti,figsize=(20,20))
    for i in range(num_category):
        for j in range(num_conti):
            axs[i,j].get_xaxis().set_visible(False)
            axs[i,j].get_yaxis().set_visible(False)
            axs[i,j].imshow(fake_x[i*num_conti+j], cmap='gray')
    plt.savefig(fig_name,bbox_inches='tight')
    plt.close()

    
#from InfoGAN import Generator, DiscriminatorFrontEnd, DiscriminatorBackend,DiscriminatorInfo
#from util import *
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

import torchvision.datasets as dset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
%matplotlib inline

BATCH_SIZE = 64
NUM_EPOCHS = 201
USE_GPU = True

DISPLAY_STEP = 100
PLOT_EPOCH = 5
#mnist_dataset = dset.MNIST(root='./data/',transform=transforms.ToTensor(),download=True)
#dataloader = DataLoader(mnist_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=1)

dataloader = train_loader
D = DiscriminatorFrontEnd()
G = Generator()
TF = DiscriminatorBackend()
Q = DiscriminatorInfo()

# Loss function
D_criterion = nn.BCEWithLogitsLoss()
Q_discr_criterion = nn.CrossEntropyLoss()
Q_conti_criterion = nn.MSELoss()


if USE_GPU:
    D = D.cuda()
    G = G.cuda()
    TF = TF.cuda()
    Q = Q.cuda()
    D_criterion = D_criterion.cuda()
    Q_discr_criterion = Q_discr_criterion.cuda()
    Q_conti_criterion = Q_conti_criterion.cuda()

optimD = optim.Adam([{'params':D.parameters()}, {'params':TF.parameters()}], lr=0.0002)
optimG = optim.Adam([{'params':G.parameters()}, {'params':Q.parameters()}], lr=0.001)

training_message = 'epoch-{:3}-step-{:3}-D_loss-{:.6f}-GQ_loss-{:.4f}'
epoch_end_message = 'epoch-{:3}-D_loss-{:.4f}-GQ_loss-{:.4f}-Image_loss-{:.4f}-Disc_loss-{:.4f}-Conti_loss-{:.4f}'

log = open('infogan.log','w')
log_message = '{:.4f},{:.4f},{:.4f},{:.4f}\n'
log.write('D_loss,G_loss,Disc_loss,Conti_loss\n')


demo_z1, demo_z2 = get_test_noise()
demo_z1 = Variable(demo_z1.cuda()) if USE_GPU else Variable(demo_z1)
demo_z2 = Variable(demo_z2.cuda()) if USE_GPU else Variable(demo_z2)


generated_images = []
psnr_score = []
mmd_score = []
fid_score = []
is_score= []
checkpoint = [1,10,20,30,40,50,100,150,200]
for epoch in range(NUM_EPOCHS):
    # Output Demo
    fake_image_array = []
    real_array = []
    if (epoch%PLOT_EPOCH) == 0:
        save_fig(demo_z1,G,'./wiegthed_z1_epoch{}.jpg'.format(epoch))
        save_fig(demo_z2,G,'./wiegthed_z2_epoch{}.jpg'.format(epoch))

    D_loss = 0 # Accumalate loss of D
    G_loss = 0 # Accumalate loss of G
    Q_loss_dis = 0 # Accumalate loss of Q (discrete)
    Q_loss_conti = 0 # Accumalate loss of Q (continuous)
    for step,batch_data in enumerate(dataloader):
        batch_size = batch_data[0].size(0)
        
        # Step 1.
        optimD.zero_grad()
        ### Real Images
        real_x = batch_data[0].cuda() if USE_GPU else batch_data[0]
        real_x = Variable(real_x)
        conv_feature_1 = D(real_x)
        prob_real = TF(conv_feature_1)
        real_label = torch.ones(batch_size).cuda() if USE_GPU else torch.ones(batch_size)
        real_label = Variable(real_label.view(-1,1),requires_grad=False)
        D_real_loss = D_criterion(prob_real,real_label)
        D_real_loss.backward()
        ### Fake Images
        z, fake_idx = sample_noise(batch_size)
        z = Variable(torch.Tensor(z).cuda()) if USE_GPU else Variable(torch.Tensor(z))
        fake_x = G(z)
        
        if epoch in checkpoint:
            generated_images.append(fake_x)
            fake_image_array.append(fake_x)
            real_array.append(real_x)
        conv_feature_2 = D(fake_x.detach())
        prob_fake = TF(conv_feature_2)
        real_label = torch.zeros(batch_size).cuda() if USE_GPU else torch.zeros(batch_size)
        real_label = Variable(real_label.view(-1,1),requires_grad=False)
        D_fake_loss = D_criterion(prob_fake,real_label)
        D_fake_loss.backward()

        D_loss += D_real_loss+D_fake_loss
        optimD.step()
        
        # Step 2.
        optimG.zero_grad()
        ### Image Reality
        conv_feature_3 = D(fake_x)
        discriminator_prediction = TF(conv_feature_3)
        fake_label = torch.ones(batch_size).cuda() if USE_GPU else torch.ones(batch_size)
        fake_label = Variable(fake_label.view(-1,1),requires_grad=False)
        generator_loss = D_criterion(discriminator_prediction,fake_label)
        G_loss += generator_loss
        ### Mutaul Info
        pred_c = Q(conv_feature_3)
        fake_idx = torch.LongTensor(fake_idx).cuda() if USE_GPU else torch.LongTensor(fake_idx)
        fake_idx = Variable(fake_idx,requires_grad=False)
        digit_classify_loss = Q_discr_criterion(pred_c[:,:10],fake_idx)
        Q_loss_dis += digit_classify_loss
        conti_loss = Q_conti_criterion(pred_c[:,10:],z[:,-2:])
        Q_loss_conti += conti_loss
        
        if epoch >= 50:
            w1 = 1.0
            w2 = 1.0
        else:
            w1 = 0.0
            w2 = 0.0
        
        G_Q_loss = generator_loss + w1*digit_classify_loss + w2*conti_loss
        G_Q_loss.backward()
        optimG.step()
        
        log.write(log_message.format(float((D_real_loss+D_fake_loss).data.cpu().numpy()),
                                     float(generator_loss.data.cpu().numpy()),
                                     float(digit_classify_loss.data.cpu().numpy()),
                                     float(conti_loss.data.cpu().numpy())))
        
        if step%DISPLAY_STEP == 0:
            print(training_message.format(epoch+1,step,float(D_loss.data.cpu().numpy())/(step+1),
                                          float((G_loss+Q_loss_dis+Q_loss_conti).data.cpu().numpy())/(step+1)),
                 flush=True,end='\r')
    # End of epoch
    D_loss = float(D_loss.data.cpu().numpy())/(step+1)
    G_loss = float(G_loss.data.cpu().numpy())/(step+1)
    Q_loss_dis = float(Q_loss_dis.data.cpu().numpy())/(step+1)
    Q_loss_conti = float(Q_loss_conti.data.cpu().numpy())/(step+1)
    print(epoch_end_message.format(epoch+1,D_loss,G_loss+Q_loss_dis+Q_loss_conti,G_loss,Q_loss_dis,Q_loss_conti))

         
    if epoch in checkpoint:
        print("========E=======P========O========C=========H=======:",epoch)
        if epoch in checkpoint:
#             temp = []
#             for batch in fake_image_array:
#                 for l in batch:
#                     temp.apped(l)
#             torch.utils.data.DataLoader(temp,shuffle=True)
            #fake_image = decoded_imgs
            #real = real_imgs
            fake_image = fake_image_array[0]
            real = real_array[0]
            psnr = []
            for i in range(len(fake_image)):
                psnr.append(PSNR(real[i],fake_image[i]))
            psnr = torch.FloatTensor(psnr)
            #PSNR_Score.append(torch.mean(psnr))              
            
            result = MMD(fake_image, real, kernel="multiscale")
            #print(f"MMD result of X and Y is {}")
        
            fretchet_dist=calculate_fretchet(real.repeat(1,3,1,1),fake_image.repeat(1,3,1,1),model)

            #print(fake.size())
            
            incp_mean,incp_std = inception_score(fake_image.repeat(1,3,1,1), cuda=True, batch_size=16, resize=True, splits=10)
            print(
            "[PSNR %f] [MMD %f] [FID: %f] [IS: %f]"
            % (torch.mean(psnr), result.item(), fretchet_dist,incp_mean)
            )
        temp_image = []
        for curr in generated_images:
            for i in curr:
                temp_image.append(i)
        trainloader = torch.utils.data.DataLoader(temp_image, batch_size=64, shuffle=True)
        find_data(trainloader,epoch)
        generated_images = []

/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


epoch-  1-D_loss-1.3097-GQ_loss-3.5566-Image_loss-0.5825-Disc_loss-2.3823-Conti_loss-0.5918
epoch-  2-D_loss-1.2489-GQ_loss-3.6017-Image_loss-0.6755-Disc_loss-2.3900-Conti_loss-0.5362
========E=======P========O========C=========H=======: 1
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 784])
torch.Size([64, 784])


/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:232: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[PSNR 56.871315] [MMD 0.128701] [FID: 209.159148] [IS: 1.798208]
epoch-  3-D_loss-1.2967-GQ_loss-3.5966-Image_loss-0.6735-Disc_loss-2.3837-Conti_loss-0.5394
epoch-  4-D_loss-1.3043-GQ_loss-3.6037-Image_loss-0.6746-Disc_loss-2.3850-Conti_loss-0.5441
epoch-  5-D_loss-1.3163-GQ_loss-3.6213-Image_loss-0.6734-Disc_loss-2.3948-Conti_loss-0.5531
epoch-  6-D_loss-1.3291-GQ_loss-3.6246-Image_loss-0.6735-Disc_loss-2.3942-Conti_loss-0.5570
epoch-  7-D_loss-1.3335-GQ_loss-3.6177-Image_loss-0.6740-Disc_loss-2.3925-Conti_loss-0.5512
epoch-  8-D_loss-1.3391-GQ_loss-3.6235-Image_loss-0.6759-Disc_loss-2.3873-Conti_loss-0.5603
epoch-  9-D_loss-1.3361-GQ_loss-3.6130-Image_loss-0.6755-Disc_loss-2.3796-Conti_loss-0.5580
epoch- 10-D_loss-1.3305-GQ_loss-3.6163-Image_loss-0.6763-Disc_loss-2.3837-Conti_loss-0.5563
epoch- 11-D_loss-1.3209-GQ_loss-3.6285-Image_loss-0.6777-Disc_loss-2.3907-Conti_loss-0.5601
========E=======P========O========C=========H=======: 10
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28,